In [270]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
import pandas as pd

After having the recepie of your model defined from all the experimentation
that you have done, it is time to encapsulate that into a neat pipeline
and use columntransformers to apply your transformations to the data.

Pipelines are usually what is used as the last version of code to be deployed,
not the notebook version

In [271]:
# load data
df = pd.read_csv('data/train.csv', index_col=0)
del df['Ticket']
del df['Cabin']
del df["Name"]

# split to X, and y
X = df.iloc[:,1:]
y = df['Survived']

In [272]:
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,male,22.0,1,0,7.2500,S
2,1,female,38.0,1,0,71.2833,C
3,3,female,26.0,0,0,7.9250,S
4,1,female,35.0,1,0,53.1000,S
5,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
887,2,male,27.0,0,0,13.0000,S
888,1,female,19.0,0,0,30.0000,S
889,3,female,NaN,1,2,23.4500,S


In [273]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 55.7+ KB


In [274]:
X.isna().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [275]:
y

PassengerId
1      0
2      1
3      1
4      1
5      0
      ..
887    0
888    1
889    0
890    1
891    0
Name: Survived, Length: 891, dtype: int64

In [276]:
type(X["Sex"].unique())

numpy.ndarray

In [277]:
# feature engineering for numericals
numeric_features = ["Fare"]


In [278]:
# create a sequential pipeline 
# output of one step will be input to the next 
numeric_transformer = make_pipeline(
    StandardScaler()
    )

In [279]:
# create a sequential pipeline 
# output of one step will be input to the next 
mbag_transformer = make_pipeline(
    SimpleImputer(strategy="most_frequent"), OneHotEncoder(handle_unknown="ignore")
    )

In [280]:
age_transformer = make_pipeline(
    SimpleImputer(strategy="mean"), StandardScaler()
    )

In [281]:
type(numeric_transformer)

sklearn.pipeline.Pipeline

In [282]:
# feature engineering for categorical
categorical_features = ["Pclass"]

# handle unknown means it will ignore nan's if it finds them
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

In [283]:
# you can also create custom functions
# df[['last_name']]
# df.columns[0] = 'last_name'
import numpy as np
def mapping_feature(df):
    mapped=df[df.columns[0]] = df[df.columns[0]].map({'female': 1, 'male': 0}).astype(int)
    return mapped.values.reshape(-1, 1)

In [284]:
mapping_feature(df[["Sex"]])



C:\Users\Master\AppData\Local\Temp/ipykernel_14528/3669140483.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapped=df[df.columns[0]] = df[df.columns[0]].map({'female': 1, 'male': 0}).astype(int)


array([[0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
    

In [285]:
# pre-process transformations
# notice here that we include a pipeline for the numeric transformations

# Columns of the original feature matrix that are not specified are dropped 
# from the resulting transformed feature matrix, unless specified in the 
# passthrough keyword

preprocessor = ColumnTransformer(
    transformers=[
        ("Age", age_transformer, ["Age"]),
        ("Embarked", mbag_transformer, ["Embarked"]),
        ("num", numeric_transformer, ["Fare"]),
        ("cat", categorical_transformer, ["Pclass"]),
        ('name', FunctionTransformer(mapping_feature), ['Sex'])
        
    ],
    remainder='passthrough')

In [294]:
"""
preprocessor.fit(X)                    # DataFrame with titanic data
Xtrain_fe = preprocessor.transform(X)  # use this to train your model
"""

'\npreprocessor.fit(X)                    # DataFrame with titanic data\nXtrain_fe = preprocessor.transform(X)  # use this to train your model\n'

In [295]:
Xtrain_fe

'Xtrain_fe'

In [288]:
type(Xtrain_fe)

numpy.ndarray

In [289]:
df_transformed=pd.DataFrame(Xtrain_fe)
df_transformed

,0,1,2,3,4,5,6,7,8,9,10
0,-0.592481,0.0,0.0,1.0,-0.502445,0.0,0.0,1.0,0.0,1.0,0.0
1,0.638789,1.0,0.0,0.0,0.786845,1.0,0.0,0.0,1.0,1.0,0.0
2,-0.284663,0.0,0.0,1.0,-0.488854,0.0,0.0,1.0,1.0,0.0,0.0
3,0.407926,0.0,0.0,1.0,0.420730,1.0,0.0,0.0,1.0,1.0,0.0
4,0.407926,0.0,0.0,1.0,-0.486337,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
886,-0.207709,0.0,0.0,1.0,-0.386671,0.0,1.0,0.0,0.0,0.0,0.0
887,-0.823344,0.0,0.0,1.0,-0.044381,1.0,0.0,0.0,1.0,0.0,0.0
888,0.000000,0.0,0.0,1.0,-0.176263,0.0,0.0,1.0,1.0,1.0,2.0
889,-0.284663,1.0,0.0,0.0,-0.044381,1.0,0.0,0.0,0.0,0.0,0.0


In [290]:
import warnings
import sklearn
import pandas as pd
def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)
    
    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
        # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                                 "provide get_feature_names. "
                                 "Will return input column names if available"
                                 % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [name + "__" + f for f in column]

        return [name + "__" + f for f in trans.get_feature_names()]
    
    ### Start of processing
    feature_names = []
    
    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == sklearn.pipeline.Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))
    
    
    for name, trans, column, _ in l_transformers: 
        if type(trans) == sklearn.pipeline.Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [name + "__" + f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))
    
    return feature_names

In [291]:
get_feature_names(preprocessor)

C:\Users\Master\AppData\Local\Temp/ipykernel_14528/1483046653.py:33: UserWarning: Transformer simpleimputer (type SimpleImputer) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "
C:\Users\Master\AppData\Local\Temp/ipykernel_14528/1483046653.py:33: UserWarning: Transformer standardscaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "
C:\Users\Master\AppData\Local\Temp/ipykernel_14528/1483046653.py:33: UserWarning: Transformer name (type FunctionTransformer) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "


['Age__Age',
 'onehotencoder__x0_C',
 'onehotencoder__x0_Q',
 'onehotencoder__x0_S',
 'num__Fare',
 'cat__x0_1',
 'cat__x0_2',
 'cat__x0_3',
 'name__Sex',
 'SibSp',
 'Parch']

In [292]:
df_transformed=pd.DataFrame(Xtrain_fe, columns=get_feature_names(preprocessor))
df_transformed

C:\Users\Master\AppData\Local\Temp/ipykernel_14528/1483046653.py:33: UserWarning: Transformer simpleimputer (type SimpleImputer) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "
C:\Users\Master\AppData\Local\Temp/ipykernel_14528/1483046653.py:33: UserWarning: Transformer standardscaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "
C:\Users\Master\AppData\Local\Temp/ipykernel_14528/1483046653.py:33: UserWarning: Transformer name (type FunctionTransformer) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "


,Age__Age,onehotencoder__x0_C,onehotencoder__x0_Q,onehotencoder__x0_S,num__Fare,cat__x0_1,cat__x0_2,cat__x0_3,name__Sex,SibSp,Parch
0,-0.592481,0.0,0.0,1.0,-0.502445,0.0,0.0,1.0,0.0,1.0,0.0
1,0.638789,1.0,0.0,0.0,0.786845,1.0,0.0,0.0,1.0,1.0,0.0
2,-0.284663,0.0,0.0,1.0,-0.488854,0.0,0.0,1.0,1.0,0.0,0.0
3,0.407926,0.0,0.0,1.0,0.420730,1.0,0.0,0.0,1.0,1.0,0.0
4,0.407926,0.0,0.0,1.0,-0.486337,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
886,-0.207709,0.0,0.0,1.0,-0.386671,0.0,1.0,0.0,0.0,0.0,0.0
887,-0.823344,0.0,0.0,1.0,-0.044381,1.0,0.0,0.0,1.0,0.0,0.0
888,0.000000,0.0,0.0,1.0,-0.176263,0.0,0.0,1.0,1.0,1.0,2.0
889,-0.284663,1.0,0.0,0.0,-0.044381,1.0,0.0,0.0,0.0,0.0,0.0


In [293]:
# create the model pipeline
pipeline2 = make_pipeline(preprocessor, LogisticRegression(max_iter=300))

# split data to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state= 42)

####
# fit the pipeline to training data
pipeline2.fit(X_train, y_train)
###
# calculate the accuracy score from test data
print("model score: %.3f" % pipeline2.score(X_test, y_test))

# get predictions from the pipeline
print(pipeline2.predict(X_test))

# get prediction probabilities from the pipeline 
print(pipeline2.predict_proba(X_test)[:, 1])

model score: 0.790
[0 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 1 1
 0 0 1 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1
 0 1 0 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 1 1 0 0
 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 1 1 0 0 0 0 0 1
 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 1
 0 0 0 1 0 1 1 1 0 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0
 1 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0]
[0.12290185 0.27717076 0.11373848 0.90754927 0.75424818 0.89647558
 0.70479943 0.08242342 0.77505428 0.8741105  0.25552678 0.06207448
 0.36944229 0.16029408 0.25217259 0.89820624 0.22979018 0.70486233
 0.30184131 0.25033227 0.1030605  0.30258193 0.5731066 

In [297]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
571,2,male,62.0,0,0,10.5000,S
788,3,male,8.0,4,1,29.1250,Q
75,3,male,32.0,0,0,56.4958,S
114,3,female,20.0,1,0,9.8250,S
636,2,female,28.0,0,0,13.0000,S
...,...,...,...,...,...,...,...
107,3,female,21.0,0,0,7.6500,S
271,1,male,NaN,0,0,31.0000,S
861,3,male,41.0,2,0,14.1083,S
